In [1]:
import pandas as pd
from stock_ROI import *

In [2]:
nasdaq_df = pd.read_csv("../Data/Polished/randomized_day_market.csv")
nasdaq_df

,Date,Day,Month,Year,Low,Open,Volume,High,Close,Ticker Symbol
0,2012-01-31,31,1,2012,8.100000,8.460000,8566500.0,8.500000,8.440000,AAL
1,2012-02-22,22,2,2012,7.690000,7.900000,5383700.0,8.080000,7.700000,AAL
2,2012-12-03,12,3,2012,6.870000,7.000000,5282400.0,7.120000,6.890000,AAL
3,2012-04-16,16,4,2012,7.850000,7.860000,7014600.0,8.170000,8.130000,AAL
4,2012-05-30,30,5,2012,12.460000,12.600000,8895400.0,12.850000,12.690000,AAL
...,...,...,...,...,...,...,...,...,...,...
185288,2021-12-31,31,12,2021,47.580002,48.150002,232200.0,48.439999,47.990002,ZUMZ
185289,2022-03-01,3,1,2022,47.820000,48.189999,289300.0,49.980000,48.009998,ZUMZ
185290,2022-03-02,3,2,2022,43.090000,43.860001,198400.0,44.900002,43.119999,ZUMZ
185291,2022-04-03,4,3,2022,43.430000,45.810001,464800.0,45.810001,44.500000,ZUMZ


In [ ]:
class portfolio():

    def __init__(self):

        # the total value of the portfolio (be noted that our portfolio only support holding ONE share per company)
        self.value = 0

        # list of all the companies that we invested in
        self.holdings = []

    def buy(self, ticker, date, price):
        pass

    def sell(self, ticker, date, price):
        pass

    def get_portfolio_value(self):
        return self.value

    def get_holdings(self):
        return self.holdings